# SSRS Expolratory Data Analysis

In [ ]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
#plt.rcParams["figure.figsize"] = (width,height)
#sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from d2e2f.visualization.visualize import plot_map, plot_trips

In [ ]:
from functools import reduce
from operator import add

In [ ]:
datas = catalog.load("ssrs.data")
data = datas['SSRS15-01_322012079']()
data.index = pd.to_datetime(data.index)

In [ ]:
data['t'] = data.index - data.index[0]
data['latitude'].notnull()

## Cleaning

In [ ]:
def quantile_clean(data: pd.DataFrame, columns :list = None, alpha=0.001):
    
    if columns is None:
        columns = data.columns
    
    for key in columns:
        values = data[key]
        mask = ((values.quantile(alpha) < values) & 
        (values.quantile(1-alpha) > values)  )
        data = data.loc[mask]
        
    return data
        

In [ ]:
columns = data.columns[~data.isnull().all()]
data_clean = data[columns]

mask = ((-90 <= data['latitude']) & (data['latitude'] <=90) &
        (0 <= data['longitude']) & (data['longitude'] <=180)     
       )



data_clean = data_clean.loc[mask]
data_clean = quantile_clean(data_clean, columns=['latitude', 'longitude'])
data_clean = data_clean.fillna(method='backfill').fillna(method='ffill')

In [ ]:
renames = {key: reduce(add, key.split('_')[2:]) for key in data_clean.columns if len(key.split('_')[2:]) > 0}
data_clean.rename(columns=renames, inplace=True)
data_clean.head()

In [ ]:
data_clean.plot(x='longitude', y='latitude')

In [ ]:
data_clean['longitude'].quantile(0.001)

In [ ]:
data_clean.plot(y='SOGdshSA13')

In [ ]:
mask = ~((data_clean.columns.str.contains('COG') | data_clean.columns.str.contains('SOG')))
columns = list(data_clean.columns[mask])
columns.remove('latitude')
columns.remove('longitude')
data_plot = data_clean[columns].resample('1D').mean()
sns.pairplot(data_plot)

In [ ]:
data_plot.sum()

In [ ]:
sns.heatmap(data_plot.corr().abs(), cmap='gray_r')

In [ ]:
rpm_columns = list(data_clean.columns[data_clean.columns.str.contains('RPM')])
fuel_rate_columns = list(data_clean.columns[data_clean.columns.str.contains('FuelRate')])


In [ ]:
data_clean['rpm'] = data_clean[rpm_columns].mean(axis=1)
data_clean['fuel_rate'] = data_clean[fuel_rate_columns].mean(axis=1)
data_new = data_clean.drop(columns = rpm_columns + fuel_rate_columns)

renames = {
'SOGdshSA13' : 'sog',
'COGTruedshSA13' : 'cog',
}
data_new.rename(columns=renames, inplace=True)

In [ ]:
sns.pairplot(data_new.resample('100S').mean().drop(columns=['latitude','longitude','cog']), kind='scatter', plot_kws={'alpha':0.01})

#g = sns.PairGrid(data_new.resample('100S').mean().drop(columns=['latitude','longitude','cog']))
#g.map_upper(sns.scatterplot)
#g.map_lower(sns.scatterplot)

In [ ]:
mask = data_new['sog']>3
data_speed = data_new.loc[mask]

In [ ]:
sns.pairplot(data_speed.drop(columns=['latitude','longitude','cog']).resample('100S').mean(), kind='scatter', plot_kws={'alpha':0.1})


In [ ]:
plot_map(df=data_speed, zoom_start=9, color_key='sog')

In [ ]:
data_new['t'] = (data_new.index-data_new.index[0]).total_seconds()
    

In [ ]:
def insert_zero_at_time_jump(data_new):
    
    mask = data_new['t'].diff() > 30
    
    assert len(data_new.loc[mask]) / len(data_new) < 10**-2
    
    jumps = data_new.loc[mask]
    
    # Insering zeros before and after a jump to get correct integration.
    for index, jump in jumps.iterrows():
        i = data_new.index.get_loc(index)
        data_new.iloc[i][['sog','fuel_rate']] = 0
        data_new.iloc[i+1][['sog','fuel_rate']] = 0
        

In [ ]:
distance = scipy.integrate.trapz(data_new['sog'], dx=1.0)
distance

In [ ]:
consumption = scipy.integrate.trapz(data_new['fuel_rate']/3600, x=data_new['t'])
consumption

In [ ]:
consumption = scipy.integrate.trapz(data_new['fuel_rate']/3600, x=data_new['t'])
consumption

In [ ]:
consumption/distance*1852*1000

In [ ]:
consumption/data_new.iloc[-1]['t']*3600*1000

In [ ]:
data_new['sog'].mean()

In [ ]:
mask = data_new['sog']>0.25
data_speed = data_new.loc[mask]
distance = scipy.integrate.trapz(data_speed['sog'], dx=1.0)
distance

In [ ]:
consumption = scipy.integrate.trapz(data_speed['fuel_rate']/3600, dx=1.0)
consumption

In [ ]:
consumption/distance*1852*1000

In [ ]:
elapsed_time = len(data_speed)*1.0
consumption/elapsed_time*3600*1000

In [ ]:
data_speed['sog'].mean()

In [ ]:
import geopandas

In [ ]:
data = geopandas.GeoDataFrame(data_speed, 
                 geometry=geopandas.points_from_xy(data_speed.longitude, data_speed.latitude, crs="EPSG:4326"))
data = data.to_crs(epsg=3006)
data['distance'] = data.distance(data.shift())

In [ ]:
data['distance'].sum()

In [ ]:
data['distance'].sum()/distance

In [ ]:
data_speed['fuel_per_m'] = data_speed['fuel_rate']/data_speed['sog']

In [ ]:
from sklearn.svm import SVR
model = LinearRegression()
mask = data_speed['sog'] > 1
data = data_speed.loc[mask].resample('30S').mean().dropna()

def features(data):
    X = pd.DataFrame(index=data.index)
    X['sog'] = data['sog']
    X['sog**2'] = data['sog']**2
    X['sog**3'] = data['sog']**3
    X['sog**4'] = data['sog']**4
    X['sog**5'] = data['sog']**5
    X['sog**6'] = data['sog']**6
    
    return X

X = features(data)
#y = data['fuel_per_m']
y = data['fuel_rate']
model.fit(X,y)

In [ ]:
X_pred = pd.DataFrame()
X_pred['sog'] = np.linspace(data_speed['sog'].min(), data_speed['sog'].max(), 100)
X_pred = features(X_pred)

prediction = X_pred.copy()
prediction['fuel_rate'] = model.predict(X_pred)
prediction['fuel_per_m'] = prediction['fuel_rate']/X_pred['sog']



In [ ]:
fig,ax=plt.subplots()
data_speed.resample('10S').mean().plot(x='sog', y='fuel_per_m', style='.', alpha=0.05, ax=ax, label='__no_label__')
ax.set_ylim(data_speed['fuel_per_m'].quantile(0.01), 
            data_speed['fuel_per_m'].quantile(0.90))
ax.plot([],'b.', label='fuel rate per m')

prediction.plot(x='sog', y='fuel_per_m', ax=ax, label='regression')
ax.set_xticks(np.arange(0,int(np.ceil(data_speed['sog'].max())),1));
ax.grid(True)
ax.legend()

In [ ]:
data_speed['sog'].hist(bins=20)

In [ ]:
model = LinearRegression(fit_intercept=True)

def add_features(X):
    X['rpm**2'] = X['rpm']**2
    #X['rpm**3'] = X['rpm']**3
    X['rpm**4'] = X['rpm']**4
    #X['rpm**5'] = X['rpm']**5
    

y = data_speed['sog'].copy()
X = data_speed[['rpm']].copy()
add_features(X)

model.fit(X=X, y=y)

In [ ]:
model.score(X, y)

In [ ]:
rpms = np.linspace(data_speed['rpm'].min(), data_speed['rpm'].max(), 100)
f = scipy.interpolate.interp1d(x=rpms, y=rpms, kind='nearest')
data_speed['rpm_round'] = f(data_speed['rpm'])
data_weighted = data_speed.groupby(by='rpm_round').mean()

In [ ]:
y2 = data_weighted['sog'].copy()
X2 = data_weighted[['rpm']].copy()
add_features(X2)
model2 = LinearRegression(fit_intercept=True)
model2.fit(X=X2, y=y2)

In [ ]:
model3 = Ridge(alpha=1000, fit_intercept=True)
model3.fit(X,y)

In [ ]:
model4 = Lasso(alpha=100, fit_intercept=True)
model4.fit(X,y)

In [ ]:
from sklearn.svm import SVR
model5 = SVR()
model5.fit(X,y)

In [ ]:
models = {'Linear1': model, 
          'Linear2': model2, 
          'Ridge': model3, 
          'Lasso': model4,
          'SVR': model5}

X_pred = pd.DataFrame()
X_pred['rpm'] = np.linspace(data_speed['rpm'].min(), data_speed['rpm'].max(), 100)
add_features(X_pred)

df_predictions = pd.DataFrame()
for name, model_ in models.items():
    df_predictions[name] = model_.predict(X_pred)

fig,ax=plt.subplots()
data_speed.plot(x='rpm', y='sog', style='.', alpha=0.01, ax=ax)

for name, y_pred in df_predictions.items():
    ax.plot(X_pred['rpm'], y_pred, label=name)

ax.legend()

In [ ]:
data_speed.plot(y='sog')

In [ ]:
data_new.plot(y='sog')

In [ ]:
data_new.sort_index(inplace=True)
data_new['t'] = (data_new.index-data_new.index[0]).total_seconds()

In [ ]:
distance = scipy.integrate.trapz(data_new['sog'], x=data_new['t'])
distance

In [ ]:
consumption = scipy.integrate.trapz(data_new['fuel_rate'], x=data_new['t'])
consumption

In [ ]:
data_new.index[-1] - data_new.index[0]

In [ ]:
consumption/distance*1852

In [ ]:
consumption/data_new.iloc[-1]['t']*3600

In [ ]:
import geopy.distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 100)

geopy.distance.geodesic(coords_1, coords_2).m

In [ ]:
geopy.distance.distance(coords_1, coords_2).m

In [ ]:
data_new['latitude']

In [ ]:
data2['t'] = (data2.index-data2.index[0]).total_seconds()
consumption = scipy.integrate.trapz(data2['fuel_rate'], x=data2['t'])

In [ ]:
consumption/distance*1852

In [ ]:
consumption/data2['t'][-1]*3600

In [ ]:
data2['sog'].mean()

In [ ]:
distance/data2.iloc[-1]['t']

In [ ]:
data2.plot(kind='line', y='sog')

In [ ]:
data_new['sog'].mean()

In [ ]:
data_new['t'].diff().plot()

In [ ]:
mask = data_new['t'].diff() > 30
data_new.loc[mask]